In [2]:
import urllib.request, json 
import pandas as pd
import numpy as np
import requests
#import pandas_profiling

* /reddit/comment/search
* /reddit/submission/search
* /reddit/subreddit/search

In [4]:

# Collection DATA with Pushshift API

with urllib.request.urlopen("https://api.pushshift.io/reddit/comment/search/?subreddit=bitcoin&after=1546318800&before=1564631940&sort_type=score&sort=desc&size=1000") as url:
    df_comment = pd.DataFrame(data = json.loads(url.read().decode())['data'])

with urllib.request.urlopen("https://api.pushshift.io/reddit/submission/search/?subreddit=bitcoin&after=1546318800&before=1564631940&sort_type=score&sort=desc&size=1000") as url:
    df = pd.DataFrame(data = json.loads(url.read().decode())['data'])



In [6]:
df.columns # author, created_utc, score, body, subreddit

Index(['all_awardings', 'author', 'author_cakeday',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'body', 'created_utc',
       'gilded', 'gildings', 'id', 'is_submitter', 'link_id', 'locked',
       'no_follow', 'parent_id', 'permalink', 'retrieved_on', 'score',
       'send_replies', 'stickied', 'subreddit', 'subreddit_id',
       'total_awards_received', 'updated_utc'],
      dtype='object')

In [8]:
def get_data(after, before):
    url = 'https://api.pushshift.io/reddit/comment/search/?size=1000&after='+str(after)+'&before='+str(before)+'&subreddit=bitcoin'
    with urllib.request.urlopen(url) as url:
        df = pd.DataFrame(data = json.loads(url.read().decode())['data'])[['created_utc','author','score','body','subreddit']]
    return df


Use to convert time: https://www.unixtimestamp.com/index.php

Jan
- After(1/1/2019) 1546300800
- Before(1/31/2019) 1548935999

Feb
- After(2/1/2019) 1548979200
- Before(2/28/2019) 1551398399

Mar
- After(3/1/2019) 1551398400
- Before(3/31/2019) 1554076799

Apr
- After(4/1/2019) 1554076800
- Before(4/30/2019) 1556668799

May
- After(5/1/2019) 1556668800
- Before(5/31/2019) 1559347199

June
- After(5/1/2019) 1559347200
- Before(5/31/2019) 1561939199



In [16]:
from datetime import date,timedelta,datetime,timestamp

ImportError: cannot import name 'timestamp' from 'datetime' (/Users/matthewlee/anaconda3/envs/metis/lib/python3.7/datetime.py)

In [35]:
import time

int(time.mktime(date(2019, 1, 1).timetuple()))

1546318800

In [17]:

datetime.timestamp(date(2019, 1, 1))

TypeError: descriptor 'timestamp' requires a 'datetime.datetime' object but received a 'datetime.date'

In [14]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

date(2019, 1, 1)
end_date = date(2019, 8, 8)
to_integer(begin_date)

20190101

In [ ]:
df = get_data(1559347200, 1561939199)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(df) > 0:
    # Calls get_data() with the created date of the last submission
    try:
        after = df['created_utc'].tail(1).item()
        df = df.append(get_data(after, 1561939199), ignore_index=True)
    except KeyError:
        break
        
print(len(df))